###  00-connectivity

In [1]:
''' ENV CONFIGURATION '''
# pip install python-dotenv
import os
from dotenv import load_dotenv

dotenv_path = os.path.join(os.getcwd(), ".env")  # Replace ".env" with the actual file path if it's in a different location
load_dotenv(dotenv_path=dotenv_path, override=True)

ACCOUNT = os.environ.get("ACCOUNT", "default")
USER = os.environ.get("USER", "default")
PASSWORD = os.environ.get("PASSWORD", "default")
ROLE = os.environ.get("ROLE", "default")
WAREHOUSE = os.environ.get("WAREHOUSE", "default")
# a3491fb2-000f-4d9f-943e-127cfe29c39c

## STEP 01 - CREATE USER AND WH 

In [3]:
from snowflake.snowpark import Session
import sys
import logging

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters =  {
       "ACCOUNT":ACCOUNT,
        "USER":USER,
        "PASSWORD":PASSWORD,
        "ROLE":ROLE,
        "DATABASE":"DEMO_DB",
        "SCHEMA":"ALF_SNOWPARK",
        "WAREHOUSE":WAREHOUSE
    }
    print(connection_parameters)
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

def main():
    session = get_snowpark_session()
    
    context_df = session.sql("select current_role(), current_database(), current_schema(), current_warehouse()")
    context_df.show(2)

    customer_df = session.sql("select c_custkey,c_name,c_phone,c_mktsegment from snowflake_sample_data.tpch_sf1.customer limit 10")
    customer_df.show(5)

if __name__ == '__main__':
    main()  


{'ACCOUNT': 'mckesson.canada-central.azure', 'USER': 'SSIS_INGEST_AU', 'PASSWORD': 'Ev?PAGOnAscT', 'ROLE': 'DEV_ENT_DW_ENGINEER_FR', 'DATABASE': 'DEMO_DB', 'SCHEMA': 'ALF_SNOWPARK', 'WAREHOUSE': 'DEMO_WH'}
08:35:43 - INFO - Snowflake Connector for Python Version: 3.7.1, Python Version: 3.8.8, Platform: Windows-10-10.0.22621-SP0
08:35:43 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
08:35:44 - INFO - Snowpark Session information: 
"version" : 1.14.0,
"python.version" : 3.8.8rc1,
"python.connector.version" : 3.7.1,
"python.connector.session.id" : 343664821690,
"os.name" : Windows

08:35:44 - INFO - query: [select current_role(), current_database(), current_schema(), current_warehouse()...]
08:35:45 - INFO - query execution done
08:35:45 - INFO - Number of results in first chunk: 0
08:35:45

## STEP 02 - CREATE SCHEMA 

In [ ]:
-- YOU EXECUTE ON FILE SQL 

create schema if not exists DEMO_DB.ALF_SOURCE; -- will have source stage etc
create schema if not exists DEMO_DB.ALF_CURATED; -- data curation and de-duplication
create schema if not exists DEMO_DB.ALF_consumption; -- fact & dimension
create schema if not exists DEMO_DB.ALF_audit; -- to capture all audit records
create schema if not exists DEMO_DB.ALF_common; -- for file formats sequence object etc

## STEP 03 - CREATE INTERNAL STAGE IN SOURCE SCHEMA 

In [ ]:
CREATE STAGE IF NOT EXISTS DEMO_DB.ALF_SOURCE.MY_INTERNAL_STAGE;